In [1]:
import pandas as pd
reviews_data = pd.read_excel('./data/Third Data.xlsx', sheet_name='Original Sheet (no breakdown)')
reviews_data = reviews_data.iloc[1:, :].drop(columns=['Column1']).rename(columns={'Column2': 'URL', 'Column3': 'review_text'}).fillna('').reset_index(drop=True)

keywords_data = pd.read_excel('./data/hugging.xlsx', sheet_name='Sheet1')

In [2]:
from review_text import ReviewText
from utils import flatten, keep_or_delete_label
from label import get_labels

labels_data, breakdowns_regardless = get_labels()

2022-06-30 15:54:08.699037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-30 15:54:08.699074: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
for lb in labels_data:
    if labels_data[lb]['type'] == 3:
        print(type(labels_data[lb]['data']))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [31]:
class CategorizeReviewTextLabel:
  def __init__(self, review_text, label):
    # create a Review Text instance
    self.review_text = ReviewText(review_text)

    self.label = label
    self.label_type = self.detect_label_type()
    if self.label_type != 0:
      self.label_data = labels_data[label]['data']
    else:
      self.label_data = {}
    self._keywords = []
    self.FOUND_KEYWORD = False

  @property
  def keywords(self):
    # flatten the keywords & remove duplicate keyword (use set) before return
    return list(set(flatten(self._keywords)))

  def get_label_data(self, key, flat=False):
    data = self.label_data[key]
    if flat:
      data = flatten(data)
      data = [value for value in data if value != '']
    return data

  def detect_label_type(self):
    if self.label == None:
      return 0
    return labels_data[self.label]['type']

  def find_keywords_no_sentiment(self, keywords, change_to_keywords):
    results = []
    for keywords_index, keywords_present in enumerate(keywords):
        for keyword in keywords_present:
          if keyword.lower() in self.review_text.text.lower():
            corresponding_keyword = change_to_keywords[keywords_index]
            results.append(corresponding_keyword)
    return results

  def add_otherwise_keyword(self):
    if 'OTHERWISE' in self.label_data:
      # get the keyword
      if len(self.label_data['OTHERWISE']) < 2:
        otherwise_keyword = self.label_data['OTHERWISE'][0]
      else:
        otherwise_keyword = self.label_data['OTHERWISE'][self.review_text.sentiment]
      self._keywords.append(otherwise_keyword)

  def processing(self):
    # label_type = 0 => not run
    if self.label_type == 0:
      self.processing_regardless()
      return

    if self.label_type == 1:
      self.processing_type_1()
    elif self.label_type == 2:
      self.processing_type_2()
    elif self.label_type == 3:
      self.processing_type_3()
    else:
      self.processing_type_4()
    self.processing_regardless()

    # if there are no keyword in the review text, add OTHERWISE keywords
    if not self.FOUND_KEYWORD:
      self.add_otherwise_keyword()

    if self.label == 'consistency and texture':
      KEEP_WORDS = flatten(self.get_label_data('UNLESS WORDS PRESENT'))
      LABELS_PRESENTS = flatten(self.get_label_data('IF LABELS PRESENT')) + flatten(breakdowns_regardless[0]['IF LABELS PRESENT'])
      keep = True

      for keyword in LABELS_PRESENTS:
        if keyword in self.keywords:
          keep = False
          break

      for word in KEEP_WORDS:
        if word in self.review_text.text.lower():
          keep = True

      if not keep:
        try:
          self._keywords.remove('consistency (positive)')
          self._keywords.remove('consistency (negative)')
        except:
          pass

  def processing_type_1(self, WORD_COMBINATION=False):
    FIND_KEYWORDS = self.get_label_data('FIND KEYWORDS')
    CORRESPONDING_KEYWORDS = self.get_label_data('RUN SENTIMENT ANALYSIS')
    OPPOSITE_WORDS = None

    for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
      for find_keyword in find_keywords:
        if find_keyword.lower() in self.review_text.text.lower():
          # find correspoding keyword
          corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index]

          # loop through all sentences and find a sentence has that keyword
          for sentence in self.review_text.sentences:
            if find_keyword.lower() in sentence.text.lower():

              # EXCEPTION: if label is fragrance and scent
              # if self.label == 'fragrance and scent':
              #   OPPOSITE_WORDS = self.label_data['OPPOSITE WORDS']
              #   for opposite_word in OPPOSITE_WORDS:
              #     k = '{0} {1}'.format(
              #       opposite_word, find_keyword.lower())
              #     if k in sentence.text.lower():
              #       self._keywords.append('fragrance free')
              
              # Exception: eye;circles;dark;darkness;bag;hood
              if WORD_COMBINATION:
                count = 0
                sentence_text = sentence.text.lower()
                for keyword in flatten(FIND_KEYWORDS):
                  count += keyword in sentence_text
                  sentence_text = sentence_text.replace(keyword, '')
                  if count >= 2:
                    break

                if count >= 2:
                  self._keywords.append(corresponding_keyword[0])

              else:
                self._keywords.append(
                  corresponding_keyword[sentence.sentiment])

              self.FOUND_KEYWORD = True

    # EXCEPTION: if label is consistency and texture
    if self.label == 'consistency and texture':
      NO_SENTIMENT_KEYWORDS = self.get_label_data('IF KEYWORDS PRESENT NO SENTIMENT', flat=True)
      for no_sentiment_keyword in NO_SENTIMENT_KEYWORDS:
        if no_sentiment_keyword in self.review_text.text.lower():
          self._keywords.append(no_sentiment_keyword)
          self.FOUND_KEYWORD = True

    # handle the exception
    if 'BREAKDOWN REGARDLESS OF SENTIMENT' in self.label_data:
      for keyword_breakdown_regardless in self.get_label_data('BREAKDOWN REGARDLESS OF SENTIMENT'):
        if keyword_breakdown_regardless in self.review_text.text.lower():
          self._keywords.append(keyword_breakdown_regardless)
    
    if 'IF WORDS PRESENT' in self.label_data:
      self._keywords.append(self.find_keywords_no_sentiment(self.get_label_data('IF WORDS PRESENT'),
                                                      self.get_label_data('CHANGE TO')))
      

  def processing_type_2(self):
    FIND_KEYWORDS = self.label_data['IF KEYWORDS PRESENT NO SENTIMENT']
    CORRESPONDING_KEYWORDS = self.label_data['CHANGE TO']
    results = self.find_keywords_no_sentiment(FIND_KEYWORDS, CORRESPONDING_KEYWORDS)
    if len(results) > 0:
      self.FOUND_KEYWORD = True
    self._keywords.append(results)

    # EXCEPTION: nightly usage
    if self.label == 'nightly usage':
      if 'overnight' in self.keywords:
        if 'night' in self.keywords:
          self._keywords = [keyword for keyword in self.keywords if keyword != 'night']

  def processing_type_3(self):
    if 'IF KEYWORDS PRESENT' not in self.label_data:
      FIND_KEYWORDS = self.label_data['RUN SENTIMENT ANALYSIS ON WHOLE REVIEW']
      keyword = FIND_KEYWORDS[self.review_text.sentiment]
      self._keywords.append(keyword)
      self.FOUND_KEYWORD = True

    else:
      TAKE_FIRST = None
      KEYWORD_FOUND = {}

      # EXCEPTION: time of usage
      if self.label == 'time of usage':
        TAKE_FIRST = True

      # handle exception
      FIND_KEYWORDS = self.label_data['IF KEYWORDS PRESENT']
      CORRESPONDING_KEYWORDS = self.label_data['ASSIGN SENTIMENT']
      for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
        for find_keyword in find_keywords:
          if find_keyword.lower() in self.review_text.text.lower():
            # find correspoding keyword
            if len(CORRESPONDING_KEYWORDS[find_keywords_index]) > 1:
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index][self.review_text.sentiment]
            else:
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index][0]

            # if EXCEPTION: time of usage, try to find first keyword
            if TAKE_FIRST:
              position_found = self.review_text.text.lower().index(find_keyword.lower())
              
              if KEYWORD_FOUND == {}:
                KEYWORD_FOUND = {'keyword': corresponding_keyword, 'position': position_found}
              else:
                if position_found < KEYWORD_FOUND['position']:
                  KEYWORD_FOUND = {'keyword': corresponding_keyword, 'position': position_found}
            else:
              self._keywords.append(corresponding_keyword)
              self.FOUND_KEYWORD = True

      if TAKE_FIRST:
        if 'keyword' in KEYWORD_FOUND:
          self._keywords.append(KEYWORD_FOUND['keyword'])
          self.FOUND_KEYWORD = True
      
      if 'IF KEYWORDS PRESENT NO SENTIMENT' in self.label_data:
        for keywords_index, keywords in enumerate(self.label_data['IF KEYWORDS PRESENT NO SENTIMENT']):
          for keyword in keywords:
            if keyword.lower() in self.review_text.text.lower():
              corresponding_keyword = self.label_data['CHANGE TO'][keywords_index]
              self._keywords.append(corresponding_keyword)
    
  def processing_type_4(self):
    CHANGE_TO = self.label_data['CHANGE TO']

    for keyword in flatten(CHANGE_TO):
      self._keywords.append(keyword)

  def processing_regardless(self):
    for idx, breakdown in enumerate(breakdowns_regardless):
      FIND_KEYWORDS = breakdown['FOR THE WORD']

      if idx == 0:
        CORRESPONDING_KEYWORDS = breakdown['RUN SENTIMENT ANALYSIS']
        OPPOSITE_WORDS = breakdown['OPPOSITE WORDS']
        
        for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
          for find_keyword in find_keywords:
            if find_keyword.lower() in self.review_text.text.lower():
              # find correspoding keyword
              
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index]

              # loop through all sentences and find a sentence has that keyword
              for sentence in self.review_text.sentences:
                if find_keyword.lower() in sentence.text.lower():
                  FIND_OPPOSITE = False
                  for opposite_word in OPPOSITE_WORDS:
                    k = '{0} {1}'.format(opposite_word, find_keyword.lower())
                    if k in sentence.text.lower():
                      self._keywords.append(corresponding_keyword[0])
                      FIND_OPPOSITE = True
                  if not FIND_OPPOSITE:
                    self._keywords.append(corresponding_keyword[sentence.sentiment])

        if 'IF LABELS PRESENT' in breakdown and 'UNLESS WORDS PRESENT' in breakdown:
          KEEP_WORDS = flatten(breakdown['UNLESS WORDS PRESENT'])
          LABELS_PRESENTS = flatten(breakdown['IF LABELS PRESENT'])

          keep = True
          
          for keyword in LABELS_PRESENTS:
            if keyword in self.keywords:
              keep = False
              break

          for word in KEEP_WORDS:
            if word in self.review_text.text.lower():
              keep = True

          if not keep:
            try:
              self._keywords.remove('consistency (positive)')
              self._keywords.remove('consistency (negative)')
            except:
              pass


      if idx == 1:
        CORRESPONDING_KEYWORDS = breakdown['RERUN CATEGORY']
        
        temp_label = self.label
        temp_label_data = self.label_data
        temp_found_keyword = self.FOUND_KEYWORD

        for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
          for find_keyword in find_keywords:
            if find_keyword.lower() in self.review_text.text.lower():
              self.label = flatten(CORRESPONDING_KEYWORDS)[find_keywords_index]
              self.label_data = labels_data[self.label]['data']
              self.FOUND_KEYWORD = False
              if find_keywords_index == 2:
                self.processing_type_1(WORD_COMBINATION=True)
              else:
                self.processing_type_1()
              if not self.FOUND_KEYWORD:
                self.add_otherwise_keyword()
        
        self.FOUND_KEYWORD = temp_found_keyword
        self.label = temp_label
        self.label_data = temp_label_data
      
      if idx == 2:
        CORRESPONDING_KEYWORDS = flatten(breakdown['RUN SENTIMENT ANALYSIS'])
        
        if self.label == 'cleansers and cleansing':
          continue

        for keyword in flatten(FIND_KEYWORDS):
          if keyword in self.review_text.text.lower():
            for sentence in self.review_text.sentences:
              if keyword.lower() in sentence.text.lower():
                self._keywords.append(CORRESPONDING_KEYWORDS[sentence.sentiment])


['thick', 'consistency (positive)']


In [ ]:
review_text = """
  I just purchased this product, and love it already. The mask can be used as a balm during the day, or at night as a mask.
  It has no residue taste, and has a nice gloss. Thick and luxurious without it seeping into the mouth.
  """
labels = ['consistency and texture']

keywords = []
for label in labels:
  categorized_review_text = CategorizeReviewTextLabel(review_text, label)
  categorized_review_text.processing()
  keywords += categorized_review_text.keywords
  print(categorized_review_text.keywords)


In [6]:
from tqdm import tqdm

# only keep url and review_text
results_df = reviews_data.drop(reviews_data.columns[2:], axis=1)
reviews_data = reviews_data.iloc[:5, :]

for idx, row in tqdm(reviews_data.iterrows()):
  url, review_text = row['URL'], row['review_text']
  review_labels = [label for label in row[2:] if isinstance(label, str)]

  if len(review_labels) == 0:
    categorized_review_text = CategorizeReviewTextLabel(review_text, None)
    categorized_review_text.processing()
    keywords.append(categorized_review_text.keywords)

  keywords = []
  for label in review_labels:
    if label in labels_data:
      categorized_review_text = CategorizeReviewTextLabel(review_text, label)
      categorized_review_text.processing()
      keywords.append(categorized_review_text.keywords)

  for idx_keyword, keyword in enumerate(list(set(flatten(keywords)))):
    keyword_column = 'Keyword {0}'.format(idx_keyword + 1)
    if keyword_column not in results_df.columns:
      results_df[keyword_column] = ''
    results_df.loc[idx, keyword_column] = keyword

0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

2022-06-22 12:47:51.224576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-22 12:47:51.224620: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-22 12:47:51.224651: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gitpodio-templatepython-v7c15bss47f): /proc/driver/nvidia/version does not exist
2022-06-22 12:47:51.224928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
5it [00:16,  3.23s/it]


In [7]:
results_df


,URL,review_text,Keyword 1,Keyword 2,Keyword 3,Keyword 4,Keyword 5,Keyword 6,Keyword 7
0,https://www.meccabeauty.co.nz/mario-badescu/hy...,Lockdown eyes are super bright. The appearan...,reduced wrinkles,shine/bright/lumi/glow/rad,,,,,
1,https://www.meccabeauty.co.nz/mario-badescu/hy...,It feels so soothing around my eyes and help...,absorb (positive),reduced wrinkles,cooling/soothing,,,,
2,https://www.meccabeauty.co.nz/mario-badescu/hy...,I bought this a couple of weeks ago along wi...,shine/bright/lumi/glow/rad,hydrating/moisturising,covered/reduced eye circles,glide makeup (positive),as/ds/pig/disc (positive),rich,weeks (positive)
3,https://www.meccabeauty.co.nz/mario-badescu/hy...,I brought this a few weeks ago and have used...,hydrating/moisturising,glide makeup (positive),weeks (positive),,,,
4,https://www.meccabeauty.co.nz/mario-badescu/hy...,This product is so nice to put on after a bi...,night,calming,shine/bright/lumi/glow/rad,days (positive),,,
...,...,...,...,...,...,...,...,...,...
995,https://www.meccabeauty.co.nz/kiehls/midnight-...,Bought this a month ago with concerns that I...,,,,,,,
996,https://www.meccabeauty.co.nz/kiehls/midnight-...,Love it's light texture that goes a long way...,,,,,,,
997,https://www.meccabeauty.co.nz/kiehls/midnight-...,This cream makes miraculous for my eyes and ...,,,,,,,
998,https://www.meccabeauty.co.nz/kiehls/midnight-...,This is probably my first ever review but I ...,,,,,,,
